In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from utils import *
pd.set_option("display.precision", 1)

### Building the Model

In [2]:
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3
num_item_features = item_train.shape[1] - 1
uvs, ivs = 3, 3
u_s, i_s = 3, 1

print_data(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,1,0,1,0,0,0,0,0,1
46970,2006,3.2,1,0,0,0,1,0,0,0,0,0,0,0,0,0
48516,2006,4.3,0,0,0,0,0,1,0,1,0,0,0,0,0,1
58559,2008,4.2,1,0,0,0,0,1,0,1,0,0,0,0,0,0


In [3]:
print("Shape of item_train:", item_train.shape)
print("Shape of user_train:", user_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of item_train: (50884, 17)
Shape of user_train: (50884, 17)
Shape of y_train: (50884,)


In [4]:
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

In [5]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)

print_data(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
1,0,-1.0,-0.8,-0.7,0.1,-0.0,-1.2,-0.4,0.6,-0.5,-0.5,-0.1,-0.6,-0.6,-0.7,-0.7
0,1,-0.7,-0.5,-0.7,-0.1,-0.2,-0.6,-0.2,0.7,-0.5,-0.8,0.1,-0.0,-0.6,-0.5,-0.4
-1,-1,-0.2,0.3,-0.4,0.4,0.5,1.0,0.6,-1.2,-0.3,-0.6,-2.3,-0.1,0.0,0.4,-0.0
0,-1,0.6,0.5,0.5,0.2,0.6,-0.1,0.5,-1.2,0.9,1.2,-2.3,-0.1,0.0,0.2,0.3
-1,0,0.7,0.6,0.5,0.3,0.5,0.4,0.6,1.0,0.6,0.3,0.8,0.8,0.4,0.7,0.7


In [6]:
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation = "relu", name = "layer1"),
    tf.keras.layers.Dense(128, activation = "relu", name = "layer2"),
    tf.keras.layers.Dense(32, activation = "linear", name = "layer3")
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation = "relu", name = "layer1"),
    tf.keras.layers.Dense(128, activation = "relu", name = "layer2"),
    tf.keras.layers.Dense(32, activation = "linear", name = "layer3")
])

class L2Normalization(keras.Layer):
    def call(self, x):
        return tf.linalg.l2_normalize(x, axis=1)

input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = L2Normalization()(vu)

input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = L2Normalization()(vm)

output = tf.keras.layers.Dot(axes=1)([vu, vm])

model = tf.keras.Model([input_user, input_item], output)
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 32)        │     40,864 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 32)        │     41,376 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l2_normalization    │ (None, 32)        │          0 │ sequential[0][0]  │
│ (L2Normalization)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l2_normalization_1  │ (None, 32)        │          0 │ sequential_1[0][… │
│ (L2Normalization)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ l2_normalization… │
│                     │                   │            │ l2_normalization… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 82,240 (321.25 KB)

 Trainable params: 82,240 (321.25 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
tf.random.set_seed(1)
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.01),
              loss = tf.keras.losses.MeanSquaredError(),
             )

In [8]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1308
Epoch 2/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1148
Epoch 3/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1112
Epoch 4/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1082
Epoch 5/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1057
Epoch 6/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1025
Epoch 7/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0992
Epoch 8/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0968
Epoch 9/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0947
Epoch 10/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0926
Epoch 11/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0907
Epoch 12/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0891
Epoch 13/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0877
Epoch 14/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0863
Epoch 15/30
1273/1273 ━━━━━━━

In [9]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - loss: 0.0887


0.08522795885801315

### Predictions for an Existing User

In [10]:
uid = 2
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

s_user_vecs = scalerUser.transform(user_vecs)
s_item_vecs = scalerItem.transform(item_vecs)

y_p = model.predict([s_user_vecs[:, u_s:], s_item_vecs[:, i_s:]])
y_pu = scalerTarget.inverse_transform(y_p)

sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()
sorted_ypu = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]
sorted_user = user_vecs[sorted_index]
sorted_y = y_vecs[sorted_index]

print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, ivs, uvs, movie_dict, maxcount = 10)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


y_p,y,user,user genre ave,movie rating ave,movie id,title,genres
4.6,5.0,2,[4.0],4.3,80906,Inside Job (2010),Documentary
4.3,4.0,2,"[4.0,4.1,3.9]",4.0,6874,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller
4.2,3.0,2,[3.9],4.0,109487,Interstellar (2014),Sci-Fi
4.2,4.0,2,"[4.0,4.1,4.0,4.0,3.9,3.9]",4.1,79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller
4.1,4.5,2,"[4.0,4.1,4.0]",4.2,58559,"Dark Knight, The (2008)",Action|Crime|Drama
4.1,4.0,2,"[4.1,4.0,3.9]",4.3,48516,"Departed, The (2006)",Crime|Drama|Thriller
4.1,4.5,2,"[4.0,4.0]",4.1,68157,Inglourious Basterds (2009),Action|Drama
4.0,3.5,2,"[4.0,4.1,4.0,3.9]",3.8,8798,Collateral (2004),Action|Crime|Drama|Thriller
4.0,5.0,2,"[4.0,4.2,3.9,3.9]",3.8,122882,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller
4.0,3.5,2,"[4.0,4.2,4.1]",4.0,91529,"Dark Knight Rises, The (2012)",Action|Adventure|Crime


### Predictions For a New User

In [11]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [12]:
user_vecs = np.tile(user_vec, (len(user_vecs), 1))

suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])
y_pu = scalerTarget.inverse_transform(y_p)

sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()
sorted_ypu = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]

print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


y_p,movie id,rating ave,title,genres
4.4,40815,3.8,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller
4.3,98809,3.8,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy
4.3,59501,3.5,"Chronicles of Narnia: Prince Caspian, The (2008)",Adventure|Children|Fantasy
4.3,5816,3.6,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
4.3,4896,3.8,Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),Adventure|Children|Fantasy
4.3,106489,3.6,"Hobbit: The Desolation of Smaug, The (2013)",Adventure|Fantasy
4.3,41566,3.4,"Chronicles of Narnia: The Lion, the Witch and the Wardrobe, The (2005)",Adventure|Children|Fantasy
4.3,54001,3.9,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy
4.2,137857,3.6,The Jungle Book (2016),Adventure|Drama|Fantasy
4.2,8368,3.9,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy
